## Load the saved model

In [5]:
from transformers import LongformerModel
import torch.nn as nn
import torch

class LongformerForRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the pretrained Longformer model (handles up to 4096 tokens)
        self.longformer = LongformerModel.from_pretrained("allenai/longformer-base-4096")
        
        # Dropout for regularization (prevents overfitting)
        self.dropout = nn.Dropout(0.1)
        
        # Final regression layer to predict 4 scores: e_score, s_score, g_score, total_score
        self.regressor = nn.Linear(self.longformer.config.hidden_size, 4)

    def forward(self, input_ids, attention_mask):
        # Forward pass through Longformer
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        
        # Try to use pooled output (if available)
        pooled_output = outputs.pooler_output
        
        # If pooled output is None (some models don't provide it), use mean pooling over all tokens
        if pooled_output is None:
            pooled_output = outputs.last_hidden_state.mean(dim=1)  # shape: (batch_size, hidden_size)
        
        # Apply dropout
        pooled_output = self.dropout(pooled_output)
        
        # Pass through regression head to get 4 continuous outputs
        return self.regressor(pooled_output)

In [7]:
model = LongformerForRegression()
output_dir = "C:/Users/steve/HuggingFace Models/LongFormer_ESG_Score"
# Load the saved weights
model.load_state_dict(torch.load(f"{output_dir}/pytorch_model.bin"))

<All keys matched successfully>

## Tokenizer

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

## Data Loader

In [8]:
from datasets import load_from_disk

tokenized_dataset= load_from_disk("esg_tokenized")

In [9]:
tokenized_dataset.set_format("torch")

In [13]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# For dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Prepare validation data in batch size of 2
test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=2, collate_fn=data_collator
)

## Testing the model

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()
loss_fn = nn.MSELoss()
test_loss = 0
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        inputs = {
            "input_ids": batch["input_ids"],
            "attention_mask": batch["attention_mask"]
        }
        labels = torch.stack([
            batch['e_score'],
            batch['s_score'],
            batch['g_score'],
            batch['total_score']
        ], dim=1).float()
        
        outputs = model(**inputs)
        loss = loss_fn(outputs, labels)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_dataloader)
print(f"Test Loss: {avg_test_loss:.4f}")

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Test Loss: 27.5613
